# About Me
* 연세대학교 전기전자공학부
* TU Delft Applied Physics 석사 과정 중
  * Realization of Kitaev Chain
<img src="delft.jpg" width="450px" height="300px"></img><br/>

In [ ]:
!pip show qiskit

In [ ]:
!pip install --upgrade qiskit

# Recap: Quantum Circuit in Qiskit
* Elements of Quantum Circuit
  1. Quantum register
  2. Classical register
  3. Quantum gates
  4. Measurements
  
<img src="stagnantWater.png" width="450px" height="300px"></img><br/>

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

In [ ]:
# Code for introduction to Qiskit syntax
qr_a = QuantumRegister(2, 'a')
qr_b = QuantumRegister(1, 'b')
cr = ClassicalRegister(2, 'c')
qc = QuantumCircuit(qr_a, qr_b, cr)
qc.x(qr_a[0])
qc.measure(qr_b[0], cr[1])
qc.measure(qr_a[1], cr[0])
qc.draw()

# Quantum Phase Estimation in Action
* T-gate : $T\equiv\begin{bmatrix}1 & 0 \\ 0 & e^{i\pi/4} \end{bmatrix}$
  * Adds phase $\pi/4$ to $|1\rangle$.
  * $|1\rangle$ is an eigenstate of T-gate.
  * Thus, we can "estimate" the phase $\theta$ s.t. $T|1\rangle=e^{2\pi i \theta}|1\rangle$.
  * Note: We know the answer already. $\theta = 0.001_{2}$
 

## Step 0: Prepare an eigenstate.
Here, the eigenstate $|\psi\rangle=|1\rangle$.   
$|\psi_0\rangle = |0\rangle^{\otimes n}|\psi\rangle$

In [ ]:
import numpy as np

In [ ]:
# Code for QPE with T-gate (θ=0.001 base 2)
qr_aux = QuantumRegister(3, 'aux')
qr_eigen = QuantumRegister(1, 'eigen')
cr = ClassicalRegister(3, 'aux_read')

qc = QuantumCircuit(qr_aux, qr_eigen, cr)

# Prepare eigenstate
qc.x(qr_eigen)
qc.barrier()
        
qc.draw()

## Step 1: Put all auxiliary qubits in superposition.
$|\psi_1\rangle=\frac{1}{2^{n/2}}(|0\rangle+|1\rangle)^{\otimes n}|\psi\rangle$

In [ ]:
# Code for QPE with T-gate (θ=0.001 base 2)
qr_aux = QuantumRegister(3, 'aux')
qr_eigen = QuantumRegister(1, 'eigen')
cr = ClassicalRegister(3, 'aux_read')

qc = QuantumCircuit(qr_aux, qr_eigen, cr)

# Prepare eigenstate
qc.x(qr_eigen)
qc.barrier()

# QPE - step 1: Superposition
qc.h(qr_aux)
qc.barrier()

qc.draw()

## Step 2: Apply controlled unitaries (T-gate) to auxilary qubits.
* Control qubit: Auxiliary quantum register. Target qubit: $|\psi\rangle$.   
* Note that the controlled unitary $U$ is applied $2^j$ times for $j^{\text{th}}$ quantum register.
  * This leads to $U^{2^j}|\psi\rangle=e^{2\pi i\cdot 2^j\theta}|\psi\rangle $
* $|\psi_2\rangle=\frac{1}{2^{n/2}}\left(|0\rangle+e^{2\pi i\theta 2^{n-1}}|1\rangle\right)\otimes \dots\otimes(|0\rangle+e^{2\pi i\theta 2^0}|1\rangle)\otimes|\psi\rangle$
* Using $k$ expressed in binary,   
$ |\psi_2\rangle=\frac{1}{2^{n/2}}\sum_{k=0}^{2^n-1}e^{2\pi i \theta k}|k\rangle\otimes|\psi\rangle$

In [ ]:
# Code for QPE with T-gate (θ=0.001 base 2)
qr_aux = QuantumRegister(3, 'aux')
qr_eigen = QuantumRegister(1, 'eigen')
cr = ClassicalRegister(3, 'aux_read')

qc = QuantumCircuit(qr_aux, qr_eigen, cr)

# Prepare eigenstate
qc.x(qr_eigen)
qc.barrier()

# QPE - step 1: Superposition
qc.h(qr_aux)
qc.barrier()

# QPE - step 2: Controlled Unibary
for idx in np.arange(len(qr_aux)):
    for digit in np.arange(2**idx): 
        qc.cp(np.pi/4, qr_aux[idx], qr_eigen)
qc.barrier()
  
qc.draw()

## Step 3: Apply inverse QFT to auxilary qubits.
* $|\psi_3\rangle=\frac{1}{2^n}\sum_x\sum_k e^{-2\pi ik(x-2^n\theta)/2^n}|x\rangle\otimes|\psi\rangle$
* $x=2^n\theta$ is the most likely outcome.

In [ ]:
def qft_dagger(qc, n):
    """n-qubit QFTdagger the first n qubits in circ"""
    # Don't forget the Swaps!
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)

In [ ]:
# Code for QPE with T-gate (θ=0.001 base 2)
qr_aux = QuantumRegister(3, 'aux')
qr_eigen = QuantumRegister(1, 'eigen')
cr = ClassicalRegister(3, 'aux_read')

qc = QuantumCircuit(qr_aux, qr_eigen, cr)

# Prepare eigenstate
qc.x(qr_eigen)
qc.barrier()

# QPE - step 1: Superposition
qc.h(qr_aux)
qc.barrier()

# QPE - step 2: Controlled Unibary
for idx in np.arange(len(qr_aux)):
    for digit in np.arange(2**idx): 
        qc.cp(np.pi/4, qr_aux[idx], qr_eigen)
qc.barrier()

# QPE - step 3: Inverse QFT
qft_dagger(qc, len(qr_aux))
qc.barrier()
        
qc.draw()

## Step 4: Measure the auxiliary qubits.
* $|\psi_4\rangle=|2^n\theta\rangle|\psi\rangle$

In [ ]:
# Code for QPE with T-gate (θ=0.001 base 2)
qr_aux = QuantumRegister(3, 'aux')
qr_eigen = QuantumRegister(1, 'eigen')
cr = ClassicalRegister(3, 'aux_read')

qc = QuantumCircuit(qr_aux, qr_eigen, cr)

# Prepare eigenstate
qc.x(qr_eigen)
qc.barrier()

# QPE - step 1: Superposition
qc.h(qr_aux)
qc.barrier()

# QPE - step 2: Controlled Unibary
for idx in np.arange(len(qr_aux)):
    for digit in np.arange(2**idx): 
        qc.cp(np.pi/4, qr_aux[idx], qr_eigen)
qc.barrier()

# QPE - step 3: Inverse QFT
qft_dagger(qc, len(qr_aux))
qc.barrier()

# QPE - step 4: Measurement
qc.measure(qr_aux, cr)
        
qc.draw()

In [ ]:
# Let's run our circuit!
# 1. Setting up fake device
from qiskit.providers.fake_provider import FakeGeneva
from qiskit_aer.noise import NoiseModel
from qiskit_aer import AerSimulator
from qiskit.tools.visualization import plot_histogram

fake_backend = FakeGeneva()  # 27-qubit device
sim = AerSimulator.from_backend(fake_backend)

In [ ]:
# 2. Running on the fake device
from qiskit import transpile

t_qc = transpile(qc, sim)
result = sim.run(t_qc).result()
counts = result.get_counts(0)
plot_histogram(counts)

In [ ]:
# 1. Setting up a real device
from qiskit_ibm_provider import IBMProvider, IBMBackend

provider = IBMProvider(instance="ibm-q-yonsei/externalq-meetup/tutorials")
backend = provider.get_backend('ibm_geneva')

In [ ]:
# 2. Running on the real device
from qiskit import transpile

t_qc = transpile(qc, backend)
result = backend.run(t_qc).result()
counts = result.get_counts(0)
plot_histogram(counts)